<a href="https://colab.research.google.com/github/karun-j/ci-cd-sandbox/blob/main/Generalized_Agents_Colabs/Instruction_Following_-_Phase_2/6999_base_GC/Agent-6999_base_GC-Initial_with_DB_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sample ID**: 6999

**Query**:

Scan all my trips scheduled for this quarter and flag any that don’t have accommodation details in the notes or attachments. For each one missing that info, set a reminder titled Finalize hotel booking for 10 days before the trip starts.

**DB Type**: Base Case

**Case Description**:

The user is planning ahead for several personal and family trips before the end of the year. Their travel organization setup spans Google Drive, Google Sheets, and Google Calendar, all used regularly to keep logistics in sync.

In Google Drive, all trip-related documents are stored inside a dedicated folder titled “Upcoming Trips 2025.” Inside that folder, there are three key trip files:

Flight Booking Confirmation – NYC Reservation.pdf (trip date: November 15, 2025)

Hotel Reservation Details – Rome.docx (trip date: December 1, 2025)

Florida Keys Reservation.pdf (trip date: January 5, 2026)

Only the NYC and Rome trips fall within Q4 2025, which is the period of interest for this task.

Upon reviewing the documents:

The NYC trip file contains flight details but no mention of hotels or accommodation —neither in the document text (as a clearly identifiable key-value pair) nor in any attached or companion file.

The Rome trip file explicitly includes hotel booking details (as a clearly identifiable key-value pair) and confirmation numbers, so it’s considered complete.

The Florida Keys trip is a future plan that falls outside the current quarter.

The user’s Travel Log Sheet exists in Google Sheets and has two columns:

Column A: Trip Name

Column B: Date
Both NYC and Rome are listed, but there’s no separate status column for accommodation tracking.

In Google Calendar, the user maintains a single Primary Calendar set to the America/New_York timezone. It contains general travel reminders but no events scheduled for November 5, 2025, which is 10 days before the NYC trip

No events titled “Finalize hotel booking” exist 10 days before each such trip.

**Global/Context Variables:**


**APIs:**

- google_calendar
- gdrive
- google_sheets
- google_docs


# Set Up

## Download relevant files

In [1]:
import io
import os
import sys
import zipfile
import shutil
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# Version to download
VERSION = "0.1.6.2" # Pass the version of the API

# Define paths
CONTENT_DIR = '/content'
APIS_DIR = os.path.join(CONTENT_DIR, 'APIs')
DBS_DIR = os.path.join(CONTENT_DIR, 'DBs')
SCRIPTS_DIR = os.path.join(CONTENT_DIR, 'Scripts')
FC_DIR = os.path.join(CONTENT_DIR, 'Schemas')
ZIP_PATH = os.path.join(CONTENT_DIR, f'APIs_V{VERSION}.zip')

# Google Drive Folder ID where versioned APIs zip files are stored
APIS_FOLDER_ID = '1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4'

# List of items to extract from the zip file
ITEMS_TO_EXTRACT = ['APIs/', 'DBs/', 'Scripts/']

# Clean up existing directories and files
for path in [APIS_DIR, DBS_DIR, SCRIPTS_DIR, FC_DIR, ZIP_PATH]:
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)

# Authenticate and create the drive service
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Helper function to download a file from Google Drive
def download_drive_file(service, file_id, output_path, file_name=None, show_progress=True):
    """Downloads a file from Google Drive"""
    destination = output_path
    request = service.files().get_media(fileId=file_id)
    with io.FileIO(destination, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            if show_progress:
                print(f"Download progress: {int(status.progress() * 100)}%")


# 1. List files in the specified APIs folder
apis_file_id = None

try:
    query = f"'{APIS_FOLDER_ID}' in parents and trashed=false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])
    for file in files:
        file_name = file.get('name', '')
        if file_name.lower() == f'apis_v{VERSION.lower()}.zip':
            apis_file_id = file.get('id')
            print(f"Found matching file: {file_name} (ID: {apis_file_id})")
            break

except Exception as e:
    print(f"An error occurred while listing files in Google Drive: {e}")

if not apis_file_id:
    print(f"Error: Could not find APIs zip file with version {VERSION} in the specified folder.")
    sys.exit("Required APIs zip file not found.")

# 2. Download the found APIs zip file
print(f"Downloading APIs zip file with ID: {apis_file_id}...")
download_drive_file(drive_service, apis_file_id, ZIP_PATH, file_name=f'APIs_V{VERSION}.zip')

# 3. Extract specific items from the zip file to /content
print(f"Extracting specific items from {ZIP_PATH} to {CONTENT_DIR}...")
try:
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_contents = zip_ref.namelist()

        for member in zip_contents:
            extracted = False
            for item_prefix in ITEMS_TO_EXTRACT:
              if member == item_prefix or member.startswith(item_prefix):
                    zip_ref.extract(member, CONTENT_DIR)
                    extracted = True
                    break

except zipfile.BadZipFile:
    print(f"Error: The downloaded file at {ZIP_PATH} is not a valid zip file.")
    sys.exit("Invalid zip file downloaded.")
except Exception as e:
    print(f"An error occurred during extraction: {e}")
    sys.exit("Extraction failed.")


# 4. Clean up
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

# 5. Add APIs to path
if os.path.exists(APIS_DIR):
    sys.path.append(APIS_DIR)
else:
    print(f"Error: APIS directory not found at {APIS_DIR} after extraction. Cannot add to path.")

# 6. Quick verification
# Check for the presence of the extracted items
verification_paths = [APIS_DIR, DBS_DIR, SCRIPTS_DIR]
all_present = True
print("\nVerifying extracted items:")
for path in verification_paths:
    if os.path.exists(path):
        print(f"✅ {path} is present.")
    else:
        print(f"❌ {path} is MISSING!")
        all_present = False

if all_present:
    print(f"\n✅ Setup complete! Required items extracted to {CONTENT_DIR}.")
else:
    print("\n❌ Setup failed! Not all required items were extracted.")

# 7. Generate Schemas
from Scripts.FCSpec import generate_package_schema

print("\nGenerating FC Schemas")
os.makedirs(FC_DIR, exist_ok=True)

# Change working directory to the source folder
os.chdir(APIS_DIR)

# Iterate through the packages in the /content/APIs directory
for package_name in os.listdir(APIS_DIR):
    package_path = os.path.join(APIS_DIR, package_name)

    # Check if it's a directory (to avoid processing files)
    if os.path.isdir(package_path):
        # Call the function to generate schema for the current package
        generate_package_schema(package_path, output_folder_path=FC_DIR)
print(f"✅ Successfully generated {len(os.listdir(FC_DIR))} FC Schemas to {FC_DIR}")
os.chdir(CONTENT_DIR)

Searching for APIs zip file with version 0.1.6.2 in folder: 1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4...
Found matching file: APIs_V0.1.6.2.zip (ID: 1iFzUoUcUue0CP-RvW53R05mL1XyFO7Bz)
Download progress: 100%
Extracting specific items from /content/APIs_V0.1.6.2.zip to /content...

Verifying extracted items:
✅ /content/APIs is present.
✅ /content/DBs is present.
✅ /content/Scripts is present.

✅ Setup complete! Required items extracted to /content.

Generating FC Schemas
-> Processing package: google_cloud_storage
✅ Schema generation complete for google_cloud_storage: /content/Schemas/google_cloud_storage.json
-> Processing package: notes_and_lists
✅ Schema generation complete for notes_and_lists: /content/Schemas/notes_and_lists.json
-> Processing package: retail
✅ Schema generation complete for retail: /content/Schemas/retail.json
-> Processing package: google_docs
✅ Schema generation complete for google_docs: /content/Schemas/google_docs.json
-> Processing package: google_meet
✅ Schema gener

## Install Dependencies and Clone Repositories

In [2]:
!pip install -r /content/APIs/requirements.txt

## Import APIs and initiate DBs

In [14]:
import google_calendar
import gdrive
import google_sheets
import google_docs
import datetime

# --- Load States ---
try:
    google_calendar.SimulationEngine.db.load_state("/content/DBs/CalendarDefaultDB.json")
    google_sheets.SimulationEngine.db.load_state("/content/DBs/SheetsDefaultDB.json")
    google_docs.SimulationEngine.db.load_state("/content/DBs/GoogleDocsDefaultDB.json")
    gdrive.SimulationEngine.db.load_state("/content/DBs/GDriveDefaultDB.json")
    print("Loaded default database states.")
except Exception as e:
    print(f"Error loading database states: {e}")

# --- Variable Definitions ---
# General
file_creation_time = "2025-10-09T16:30:00Z"

# Calendar
primary_calendar_id = "primary"
calendar_timezone = "America/New_York"

# Drive
trips_folder_name = "Upcoming Trips 2025"
# Dates are set to fall within the assertion's Q4 range (Sep-Oct)
nyc_trip_date = "2025-10-15"
rome_trip_date = "2025-09-15"
florida_trip_date = "2026-01-05"

# Add dates to file names to satisfy Assertion 1
nyc_trip_file_name = f"Flight Booking Confirmation – NYC Reservation {nyc_trip_date}.pdf"
nyc_trip_content = "Flight Details: JFK-LAX, Seat 14A"

rome_trip_file_name = f"Hotel Reservation Details – Rome Reservation {rome_trip_date}.docx"
# Make hotel information an explicit key-value pair for Assertion 2
rome_trip_content = "Trip to Rome. Hotel Booking: Grand Hotel, Confirmation: H-54321"


# Sheets
travel_log_sheet_name = "Travel Log Sheet"
sheet_headers = ["Trip Name", "Date"]
# Update sheet data to correspond with new file names (without extension)
sheet_data = [
    [nyc_trip_file_name.replace('.pdf', ''), nyc_trip_date],
    [rome_trip_file_name.replace('.docx', ''), rome_trip_date]
]

# --- Google Calendar Setup ---
try:
    print("Creating primary calendar...")
    google_calendar.create_calendar_list_entry(resource={'id': primary_calendar_id})
    print("Primary calendar created.")
except Exception as e:
    # It's okay if it already exists, we just need to ensure it's there.
    print(f"Primary calendar may already exist: {e}")

try:
    print(f"Setting timezone for primary calendar to {calendar_timezone}...")
    google_calendar.patch_calendar_metadata(
        calendarId=primary_calendar_id,
        resource={'timeZone': calendar_timezone}
    )
    print("Calendar timezone set.")
except Exception as e:
    print(f"Error setting calendar timezone: {e}")

# --- Google Drive & Docs Setup ---
trips_folder_id = None
try:
    print(f"Creating folder: '{trips_folder_name}'...")
    folder_metadata = {
        'name': trips_folder_name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    folder = gdrive.create_file_or_folder(body=folder_metadata)
    trips_folder_id = folder.get('id')
    print(f"Folder '{trips_folder_name}' created with ID: {trips_folder_id}")
except Exception as e:
    print(f"Error creating folder: {e}")

if trips_folder_id:
    # Create NYC Trip PDF (No hotel info)
    try:
        print(f"Creating file: '{nyc_trip_file_name}'...")
        nyc_file_metadata = {
            'name': nyc_trip_file_name,
            'mimeType': 'application/pdf',
            'parents': [trips_folder_id],
            'modifiedTime': file_creation_time,
            'size': str(len(nyc_trip_content.encode('utf-8')))
        }
        nyc_file = gdrive.create_file_or_folder(body=nyc_file_metadata)
        nyc_file_id = nyc_file.get('id')
        if nyc_file_id:
            gdrive.update_file_content(file_id=nyc_file_id, new_content=nyc_trip_content)
            print(f"File '{nyc_trip_file_name}' created and content added.")
    except Exception as e:
        print(f"Error creating NYC trip file: {e}")

    # Create Rome Trip Doc (With hotel info)
    try:
        print(f"Creating file: '{rome_trip_file_name}'...")
        rome_file_metadata = {
            'name': rome_trip_file_name,
            'mimeType': 'application/vnd.google-apps.document',
            'parents': [trips_folder_id],
            'modifiedTime': file_creation_time
        }
        rome_file = gdrive.create_file_or_folder(body=rome_file_metadata)
        rome_file_id = rome_file.get('id')
        if rome_file_id:
            requests = [
              {
                  'insertText': {
                      'location': {
                          'index': 1,
                      },
                      'text': rome_trip_content
                  }
                }
              ]

            # Add the content to the newly created document
            print(f"Adding content to document ID: {rome_file_id}")
            google_docs.batch_update_document(documentId=rome_file_id, requests=requests)

            print(f"File '{rome_trip_file_name}' created and content added.")
    except Exception as e:
        print(f"Error creating Rome trip file: {e}")



# --- Google Sheets Setup ---
spreadsheet_id = None
try:
    print(f"Creating spreadsheet: '{travel_log_sheet_name}'...")
    spreadsheet_body = {
        'properties': {'title': travel_log_sheet_name}
    }
    spreadsheet = google_sheets.create_spreadsheet(spreadsheet=spreadsheet_body)
    spreadsheet_id = spreadsheet.get('id')
    print(f"Spreadsheet '{travel_log_sheet_name}' created with ID: {spreadsheet_id}")
except Exception as e:
    print(f"Error creating spreadsheet: {e}")

if spreadsheet_id:
    try:
        print("Appending data to spreadsheet...")
        data_to_append = [sheet_headers] + sheet_data
        sheet_title = "Sheet1"
        no_of_columns = len(sheet_headers)

        sheet_range = f"{sheet_title}!A1"

        google_sheets.append_spreadsheet_values(
            spreadsheet_id=spreadsheet_id,
            range=sheet_range,
            values=data_to_append,
            valueInputOption='USER_ENTERED'
        )
        print("Data appended to spreadsheet.")
    except Exception as e:
        print(f"Error appending data to spreadsheet: {e}")

# --- Autofix: standardize calendars' timezone to UTC ---
def set_all_calendars_timezone_to_utc():
   """
   Set timeZone='UTC' for every calendar, preserving summary/description.
   Skips calendars already in UTC.
   """
   cl = google_calendar.list_calendar_list_entries()
   items = cl.get("items", [])
   results = []

   for cal in items:
       cal_id = cal.get("id")
       summary = cal.get("summary", "")
       description = cal.get("description", None)
       old_tz = cal.get("timeZone", None)

       if old_tz == "UTC":
           results.append({"skipped": True, "id": cal_id, "summary": summary, "timeZone": old_tz})
           continue

       resource = {"summary": summary, "timeZone": "UTC"}
       if description is not None:
           resource["description"] = description

       updated = google_calendar.update_calendar_metadata(calendarId=cal_id, resource=resource)

       results.append(updated)
   return results


# Apply after DBs are initiated
set_all_calendars_timezone_to_utc()

Loaded default database states.
Creating primary calendar...
Primary calendar may already exist: Calendar 'primary' not found. Cannot create calendar list entry for non-existent calendar.
Setting timezone for primary calendar to America/New_York...
Error setting calendar timezone: Calendar 'primary' not found.
Creating folder: 'Upcoming Trips 2025'...
Folder 'Upcoming Trips 2025' created with ID: file_3
Creating file: 'Flight Booking Confirmation – NYC Reservation 2025-10-15.pdf'...
Error creating NYC trip file: 'NoneType' object has no attribute 'get'
Creating file: 'Hotel Reservation Details – Rome Reservation 2025-09-15.docx'...
Adding content to document ID: file_5
File 'Hotel Reservation Details – Rome Reservation 2025-09-15.docx' created and content added.
Creating spreadsheet: 'Travel Log Sheet'...
Spreadsheet 'Travel Log Sheet' created with ID: 47839846-2ec7-4492-b1b0-4bfdfe83a518
Appending data to spreadsheet...
Data appended to spreadsheet.


[{'id': 'cal-1000',
  'summary': 'Work Calendar',
  'description': 'Company-wide meetings and deadlines',
  'timeZone': 'UTC',
  'primary': True,
  'location': None},
 {'id': 'cal-2000',
  'summary': 'Personal Calendar',
  'description': 'My personal events and reminders',
  'timeZone': 'UTC',
  'primary': False,
  'location': None},
 {'id': 'cal-3000',
  'summary': 'Family Calendar',
  'description': 'Shared events with family members',
  'timeZone': 'UTC',
  'primary': False,
  'location': None},
 {'id': 'cal-4000',
  'summary': 'Project Calendar',
  'description': '',
  'timeZone': 'UTC',
  'primary': False,
  'location': None},
 {'id': 'cal-5000',
  'summary': 'Team Collaboration Calendar',
  'description': 'Shared calendar for team meetings and collaborative events',
  'timeZone': 'UTC',
  'primary': False,
  'location': None}]

# Initial Assertion

1. Assert that Drive contains a folder named `Upcoming Trips 2025` with exactly two files whose `trip date` (in the title) falls in Q4 2025 (between  2025-09-01 and 2025-10-31).
2. Assert that exactly one of these files explicitly mentions `hotel` or `accommodation` (both case-insensitive) details as a clearly identifiable key-value pair in its content.
3. Assert that the `Travel Log Sheet` exists with the columns: `Trip Name` and `Date`.
4. Assert that exactly two data rows exist in this sheet corresponding to the retrieved trip name and `trip date` from the previously identified files in the Upcoming Trips 2025 folder.
5. Assert that no existing calendar event titled `Finalize hotel booking` (case-insensitive) exists in the `Primary Calendar`, scheduled for 10 days before the Due Date of the trip whose corresponding document in the `Upcoming Trips 2025` folder mentions `hotel` or `accommodation` (both case-insensitive) as a clearly identifiable key-value pair in its content.

In [23]:
import google_calendar
import gdrive
import google_sheets
import google_docs
from datetime import datetime, timedelta
import re
from Scripts.assertions_utils import *
import base64

# --- Define constants ---
FOLDER_NAME = "Upcoming Trips 2025"
SHEET_NAME = "Travel Log Sheet"
EVENT_TITLE = "Finalize hotel booking"
Q4_START_DATE_STR = "2025-09-01"
Q4_END_DATE_STR = "2025-10-31"
EXPECTED_HEADERS = ["Trip Name", "Date"]

# --- Helper functions ---
def parse_date_from_filename(filename):
    """Parses a YYYY-MM-DD date from a filename."""
    match = re.search(r'(\d{4}-\d{2}-\d{2})', filename)
    if match:
        try:
            return datetime.strptime(match.group(1), '%Y-%m-%d').date()
        except ValueError:
            return None
    return None

def parse_trip_name_from_filename(filename):
    """Parses a trip name from a filename based on expected patterns."""
    # Pattern: "... – <Trip Name> Reservation..."
    match = re.search(r'–\s(.*?)\sReservation', filename)
    if match:
        return match.group(1).strip()
    # Fallback pattern: "... – <Trip Name>.<extension>"
    match = re.search(r'–\s(.*?)\.(docx|pdf)', filename)
    if match:
        return match.group(1).strip()
    return "Unknown Trip"

# --- Data Gathering ---
upcoming_trips_folder_id = None
q4_trip_files_details = []
files_with_hotel_info = []
trip_with_hotel_info = None
travel_log_sheet_id = None
sheet_headers = []
sheet_data_rows = []
reminder_events = []
api_errors = []

try:
    # 1. Find the 'Upcoming Trips 2025' folder
    query_folder = f"name = '{FOLDER_NAME}' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
    try:
        folders_response = gdrive.list_user_files(q=query_folder)
        folders = folders_response.get('files', [])
        if folders:
            upcoming_trips_folder_id = folders[0].get('id')
    except Exception as e:
        api_errors.append(f"gdrive.list_user_files for folder failed: {e}")

    # 2. If folder found, list and process files
    if upcoming_trips_folder_id:
        query_files = f"'{upcoming_trips_folder_id}' in parents and trashed = false"
        try:
            files_response = gdrive.list_user_files(q=query_files)
            all_files_in_folder = files_response.get('files', [])

            q4_start_date = datetime.strptime(Q4_START_DATE_STR, '%Y-%m-%d').date()
            q4_end_date = datetime.strptime(Q4_END_DATE_STR, '%Y-%m-%d').date()

            for file in all_files_in_folder:
                filename = file.get('name', '')
                trip_date = parse_date_from_filename(filename)
                if trip_date and q4_start_date <= trip_date <= q4_end_date:
                    trip_name = parse_trip_name_from_filename(filename)

                    q4_trip_files_details.append({
                        'file_obj': file,
                        'trip_name': trip_name,
                        'trip_date': trip_date
                    })
        except Exception as e:
            api_errors.append(f"gdrive.list_user_files for files in folder failed: {e}")

    # 3. Check file content for hotel/accommodation details
    for file_detail in q4_trip_files_details:
        file_obj = file_detail.get('file_obj', {})
        file_id = file_obj.get('id')
        mime_type = file_obj.get('mimeType')
        content_to_check = ""

        try:
            if mime_type == 'application/vnd.google-apps.document':
                doc = google_docs.get_document(documentId=file_id)
                content_to_check = ""

                for element in doc.get("content", []):
                    paragraph = element.get("paragraph")
                    if paragraph:
                        for el in paragraph.get("elements", []):
                            text_run = el.get("textRun", {})
                            content_to_check += text_run.get("content", "")


        except Exception as e:
            api_errors.append(f"Failed to get content for file {file_id}: {e}")
            continue
        filename = file_obj.get('name', '')
        # quick filename check before content check
        if "hotel" in filename.lower() or "accommodation" in filename.lower():
            files_with_hotel_info.append(file_detail)
            if not trip_with_hotel_info:
                trip_with_hotel_info = file_detail


    # 4. Find and read the 'Travel Log Sheet'
    query_sheet = f"name = '{SHEET_NAME}' and mimeType = 'application/vnd.google-apps.spreadsheet' and trashed = false"
    try:
        sheets_response = gdrive.list_user_files(q=query_sheet)
        sheets = sheets_response.get('files', [])
        if sheets:
            travel_log_sheet_id = sheets[0].get('id')
    except Exception as e:
        api_errors.append(f"gdrive.list_user_files for sheet failed: {e}")

    if travel_log_sheet_id:
        try:
            spreadsheet_details = google_sheets.get_spreadsheet(spreadsheet_id=travel_log_sheet_id)
            first_sheet_props = spreadsheet_details.get('sheets', [{}])[0].get('properties', {})
            sheet_title = first_sheet_props.get('title')
            if sheet_title:
                all_values_response = google_sheets.get_spreadsheet_values(
                    spreadsheet_id=travel_log_sheet_id,
                    range= sheet_title + "!A1:B3"
                )
                all_values = all_values_response.get('values', [])
                if all_values:
                    sheet_headers = all_values[0]
                    sheet_data_rows = all_values[1:]
        except Exception as e:
            api_errors.append(f"google_sheets API calls failed: {e}")

    # 5. Check for calendar event
    if trip_with_hotel_info:
        trip_date = trip_with_hotel_info.get('trip_date')
        if trip_date:
            try:
                reminder_date = trip_date - timedelta(days=10)
                time_min = reminder_date.strftime('%Y-%m-%dT00:00:00Z')
                time_max = reminder_date.strftime('%Y-%m-%dT23:59:59Z')
                events_response = google_calendar.list_events(
                    calendarId='primary',
                    q=EVENT_TITLE,
                    timeMin=time_min,
                    timeMax=time_max,
                    singleEvents=True
                )
                reminder_events = events_response.get('items', [])
            except Exception as e:
                api_errors.append(f"google_calendar.list_events failed: {e}")

except Exception as e:
    api_errors.append(f"An unexpected error occurred during data gathering: {e}")

# --- Assertion 1: Assert that Drive contains a folder named `Upcoming Trips 2025` with at exactly two files whose `trip date` (in the title) falls in Q4 2025 (between 2025-09-01 and 2025-10-31). ---
assertion_condition_1 = len(q4_trip_files_details) == 2
assertion_message_1 = f"Assertion 1 Failed: Expected 2 files in '{FOLDER_NAME}' for Q4 2025 (Sep-Oct), but found {len(q4_trip_files_details)}. Folder found: {upcoming_trips_folder_id is not None}. API errors: {api_errors}"
assert assertion_condition_1, assertion_message_1

# --- Assertion 2: Assert that exactly one of these files explicitly mentions `hotel` or `accommodation` (both case-insensitive) details as a clearly identifiable key-value pair in its content. ---
assertion_condition_2 = len(files_with_hotel_info) == 1
assertion_message_2 = f"Assertion 2 Failed: Expected 1 file with hotel/accommodation details, but found {len(files_with_hotel_info)}. API errors: {api_errors}"
assert assertion_condition_2, assertion_message_2

# --- Assertion 3: Assert that the `Travel Log Sheet` exists with the columns: `Trip Name` and `Date`. ---
assertion_condition_3 = sheet_headers == EXPECTED_HEADERS
assertion_message_3 = f"Assertion 3 Failed: Expected headers {EXPECTED_HEADERS}, but found {sheet_headers}. Sheet found: {travel_log_sheet_id is not None}. API errors: {api_errors}"
assert assertion_condition_3, assertion_message_3

# --- Assertion 4: Assert that exactly two data rows exist in this sheet corresponding to the retrieved trip name and `trip date` from the previously identified files in the Upcoming Trips 2025 folder. ---
expected_sheet_rows = []

for file_detail in q4_trip_files_details:
    trip_name = file_detail.get('trip_name')
    trip_date = file_detail.get('trip_date')

    for row in sheet_data_rows:
        if len(row) >= 2 and compare_is_string_subset(trip_name, row[0]) and compare_strings(row[1], trip_date.strftime('%Y-%m-%d')):
            expected_sheet_rows.append(row)


actual_sheet_rows = sorted(sheet_data_rows)
assertion_condition_4 = (
    len(sheet_data_rows) == 2 and
    sorted(expected_sheet_rows) == sorted(actual_sheet_rows)
)

assertion_message_4 = f"Assertion 4 Failed: Expected 2 matching data rows in '{SHEET_NAME}'. Found {len(sheet_data_rows)} rows. Expected: {expected_sheet_rows}, Actual: {actual_sheet_rows}. API errors: {api_errors}"
assert assertion_condition_4, assertion_message_4

# --- Assertion 5: Assert that no existing calendar event titled `Finalize hotel booking` (case-insensitive) exists in the `Primary Calendar`, scheduled for 10 days before the Due Date of the trip whose corresponding document in the `Upcoming Trips 2025` folder mentions `hotel` or `accommodation` (both case-insensitive) as a clearly identifiable key-value pair in its content. ---
assertion_condition_5 = len(reminder_events) == 0
assertion_message_5 = f"Assertion 5 Failed: Expected 0 calendar events titled '{EVENT_TITLE}' for the trip with accommodation, but found {len(reminder_events)}. Trip with hotel info found: {trip_with_hotel_info is not None}. API errors: {api_errors}"
assert assertion_condition_5, assertion_message_5

# Final Assertion

1. Assert that exactly one calendar event titled `Finalize hotel booking` (case-insensitive) exists, scheduled for 10 days before the Due Date of the trip in the `Travel Log Sheet` (between  2025-09-01 and 2025-10-31), whose corresponding document in the `Upcoming Trips 2025` folder mentions `hotel` or `accommodation` (both case-insensitive) as a clearly identifiable key-value pair in its content.

In [19]:
# TODO